### Сторонние библиотеки использовать нельзя

### Задача 0 [Библиотека] (0.15 балла)  

**Условие:** 


В библиотеке хранятся книги и журналы. У каждой сущности есть общие характеристики, такие как: название, автор, жанр, число страниц, формат страниц, индекс редкости (от 1 до 10) и текст. Также у разных сущностей могут быть свои атрибуты. Хочется все редкие издания (индекс 9 или 10) дополнительно сохранять в некое хранилище (пусть json-файл), а также хочется понимать какую площадь занимает издание, если разложить все его страницы на полу.     


**Комментарий:**

Это задача с семинара на организацию иерархии классов. Идея в том, что нужно разделять сущности в зависимости от их применения. Например, есть книга как некий абстрактный объект, а есть библиотечная книга, у которой есть свои особенности. Также для сохранения книг в json нужно использвать классы-примеси.


Иерархия классов:

In [1]:
import json

PAGES_FORMAT = {
    'A1': (2048, 1024),
    'A2': (1024, 512),
    'A3': (512, 256),
    'A4': (297, 210),
}

rare_editions_filepath = "rare_editions.jsonl"


class ReadableEntity:
    '''
    A class used to represent Readable Entity
    '''
    def __init__(self, name, genre, num_pages, format_pages, index_rareness, text):
        """
        :param name: name of the ReadableEntity, len > 0
        :type name: str
        
        :param genre: genre of the ReadableEntity, len > 0
        :type genre: str

        :param num_pages: number of pages of the ReadableEntity, greater than 0
        :type num_pages: int

        :param format_pages: format of pages of the ReadableEntity, one of the PAGES_FORMAT
        :type format_pages: str

        :param index_rareness: index of rareness of the ReadableEntity, integer in range [1,10]
        :type index_rareness: int

        :param text: text of the ReadableEntity, len > 0
        :type text: str
        """
        try:
            num_pages = int(num_pages)
            index_rareness = int(index_rareness)
        except ValueError:
            print("Wrong number type")
            raise

        for param in [num_pages, index_rareness]:
            if not isinstance(param, int):
                raise TypeError("Wrong number type for", param)

        for param in [name, genre, format_pages, text]:
            if not isinstance(param, str):
                raise TypeError("Wrong number type for", param)

        if num_pages <= 0:
            raise ValueError("num_pages should be positive")

        if index_rareness not in range(1, 11):
            raise ValueError("index_rareness should be integer number in [1,10]")

        if len(name) == 0 or len(genre) == 0 or len(text) == 0:
            raise Exception("String parameters should not be empty")

        self.name = name
        self.genre = genre
        self.num_pages = num_pages
        self.format_pages = format_pages
        self.index_rareness = index_rareness
        self.text = text


class Journal(ReadableEntity):
    '''
    A class used to represent a real Journal as an ancestor of ReadableEntity
    '''
    def __init__(self, name, genre, num_pages, format_pages, index_rareness, text, has_horoscope=None):
        '''
        :param has_horoscope: flag of horoscope, optional
        :type has_horoscope: bool
        '''
        super().__init__(name, genre, num_pages, format_pages, index_rareness, text)
        
        if has_horoscope != None:
            if has_horoscope == 0 or has_horoscope == 1:
                has_horoscope = bool(has_horoscope)
            if not isinstance(has_horoscope, bool):
                raise TypeError("Wrong number type for", has_horoscope)

        self.has_horoscope = has_horoscope


class Book(ReadableEntity):
    '''
    A class used to represent a real Book as an ancestor of ReadableEntity
    '''
    def __init__(self, name, genre, num_pages, format_pages, index_rareness, text, has_authors_preface=None,
                 has_authors_afterword=None):
        '''
        :param has_authors_preface: flag of author's preface, optional
        :type has_authors_preface: bool
        
        :param has_authors_afterword: flag of author's afterword, optional
        :type has_authors_afterword: bool
        '''
        super().__init__(name, genre, num_pages, format_pages, index_rareness, text)
        
        for param in [has_authors_preface, has_authors_afterword]:
            if param != None:
                if param == 0 or param == 1:
                    param = bool(param)
                if not isinstance(param, bool):
                    raise TypeError("Wrong number type for", param)

        self.has_authors_preface = bool(has_authors_preface)
        self.has_authors_afterword = bool(has_authors_afterword)


class LibraryEntity:
    '''
    A class used to represent Entity in a Library
    '''
    def __init__(self, is_available_now=None, last_user_name=None):
        '''
        :param is_available_now: flag of availability in library, optional
        :type is_available_now: bool
        
        :param last_user_name: name of last user, optional
        :type last_user_name: str
        '''
        if is_available_now != None:
            if is_available_now == 0 or is_available_now == 1:
                is_available_now = bool(is_available_now)
            if not isinstance(is_available_now, bool):
                raise TypeError("Wrong number type for", is_available_now)

        if last_user_name != None:
            if not isinstance(last_user_name, str):
                raise TypeError("Wrong number type for", last_user_name)

        if len(last_user_name) == 0:
            raise Exception("last_user_name should not be empty")

        self.is_available_now = is_available_now
        self.last_user_name = last_user_name


class Exporter:
    '''
    A class used to represent Exporter 
    '''
    def export_to_txt(self, file_path):
        '''
        Exports info about oblect to txt
        '''
        with open(file_path, 'w') as f:
            for key in self.__dict__:
                f.write("{}: {}\n".format(key, self.__dict__[key]))
            f.write('\n')

    def export_to_json(self, filepath="rare_editions.jsonl"):
        with open(filepath, "a+", encoding="utf-8") as f:
            f.write(json.dumps(self.__dict__, ensure_ascii=False) + "\n")
            

class SpaceCalculator:
    '''
    A class used to represent Calculator of the Space
    '''
    def calculate_space(self):
        '''
        Calculates the space of pages according to PAGES_FORMAT
        
        :return: the space of pages 
        :type: int
        '''
        height, weight = PAGES_FORMAT[self.format_pages]
        return height * weight * self.num_pages


class LibraryJournal(Journal, LibraryEntity):
    '''
    A class used to represent a Journal in the Library, an ancestor of Journal and LibraryEntity
    '''
    def __init__(self, name, genre, num_pages, format_pages, index_rareness, text, has_horoscope=None,
                 is_available_now=None, last_user_name=None):
        Journal.__init__(self, name, genre, num_pages, format_pages, index_rareness, text, has_horoscope)
        LibraryEntity.__init__(self, is_available_now, last_user_name)
        

class LibraryBook(Book, LibraryEntity):
    '''
    A class used to represent a Book in the Library, an ancestor of Book and LibraryEntity
    '''
    def __init__(self, name, genre, num_pages, format_pages, index_rareness, text, has_authors_preface=None,
                 has_authors_afterword=None, is_available_now=None, last_user_name=None):
        Book.__init__(self, name, genre, num_pages, format_pages, index_rareness, text, has_authors_preface,
                      has_authors_afterword)
        LibraryEntity.__init__(self, is_available_now, last_user_name)


class LibraryJournal(LibraryJournal, Exporter, SpaceCalculator):
    '''
    A mix-in class used to represent class LibraryJournal with the functionality of Exporter and SpaceCalculator classes
    '''
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        if self.index_rareness in [9, 10]:
            self.export_to_json()
    

class LibraryBook(LibraryBook, Exporter, SpaceCalculator):
    '''
    A mix-in class used to represent class LibraryBook with the functionality of Exporter and SpaceCalculator classes
    '''
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        if self.index_rareness in [9, 10]:
            self.export_to_json()

In [2]:
cosmo = LibraryJournal(
    name='First', 
    genre='Entertaining', 
    num_pages=100, 
    format_pages='A4', 
    index_rareness=10, 
    text='Ну приветик!',
    has_horoscope=1.,
    is_available_now=True,
    last_user_name='IRAAAAA'
)

print(cosmo.index_rareness, cosmo.is_available_now, cosmo.last_user_name, cosmo.has_horoscope)
print(cosmo.calculate_space())

10 True IRAAAAA True
6237000


In [3]:
cosmo2 = LibraryJournal(
    name='Cosmo2', 
    genre='Entertaining', 
    num_pages=100, 
    format_pages='A4', 
    index_rareness=9, 
    text='Ну приветик!',
#     has_horoscope=True,
#     is_available_now=True,
    last_user_name='IRAAAAA'
)

print(cosmo2.has_horoscope, cosmo2.is_available_now, cosmo2.last_user_name)
print(cosmo2.calculate_space())

None None IRAAAAA
6237000


In [4]:
cosmo3 = LibraryJournal(
    name='Cosmo3', 
    genre='Entertaining', 
    num_pages=100, 
    format_pages='A4', 
    index_rareness=2, 
    text='Ну приветик!',
    has_horoscope=1.,
    is_available_now=True,
    last_user_name='IRAAAAA'
)

print(cosmo3.index_rareness, cosmo3.is_available_now, cosmo3.last_user_name, cosmo3.has_horoscope)
print(cosmo3.calculate_space())

2 True IRAAAAA True
6237000


In [5]:
pushkin = LibraryBook(
    name='Onegin', 
    genre='Drama', 
    num_pages=100, 
    format_pages='A2', 
    index_rareness=9, 
    text='Ну приветик! Onegin',
    has_authors_preface=1.,
    has_authors_afterword=False,
    is_available_now=False,
    last_user_name='Max'
)

print(pushkin.has_authors_preface, pushkin.is_available_now, pushkin.last_user_name)
print(pushkin.calculate_space())

True False Max
52428800


Прочитаем файл с редкими изданиями:

In [6]:
with open(rare_editions_filepath, "r", encoding="utf-8") as r:
    print(r.read())

{"name": "First", "genre": "Entertaining", "num_pages": 100, "format_pages": "A4", "index_rareness": 10, "text": "Ну приветик!", "has_horoscope": true, "is_available_now": true, "last_user_name": "IRAAAAA"}
{"name": "Cosmo2", "genre": "Entertaining", "num_pages": 100, "format_pages": "A4", "index_rareness": 9, "text": "Ну приветик!", "has_horoscope": null, "is_available_now": null, "last_user_name": "IRAAAAA"}
{"name": "Onegin", "genre": "Drama", "num_pages": 100, "format_pages": "A2", "index_rareness": 9, "text": "Ну приветик! Onegin", "has_authors_preface": true, "has_authors_afterword": false, "is_available_now": false, "last_user_name": "Max"}



### Задача 1 [Размер объектов] (0 - 0.15 балла)  

**Условие:** 

Написать функцию получения реального объема занимаемой объектом памяти объектом. 


1) Для int, str, list, tuple, dict **(0.05 балла)**

2) Для всех типов **(+0.1 балла)**


**Комментарий:**

На занятиях не раз говорилось, что `sys.getsizeof` умеет находить размер простых объектов, но если речь идет об объектах, вроде list, то функция вернет не совсем то, что может ожидать разработчик, потому что список хранит указатели на объекты. 

*Пример:*
```
sys.getsizeof([]) == 64
sys.getsizeof(['aaaaaaa']) == 72
```
Но
```
sys.getsizeof('aaaaaaa') == 56
```


In [7]:
import sys


def my_getsizeof(obj, measured=None):
    '''
    Calculates actual size of objects 
        
        :param obj: object to measure size
        :type obj: any type
        
        :param measured: set of measured ids
        :type measured: set
    
        :obj_size: the size of object
        :obj_size: int
    '''

    if measured is None:
        measured = set()

    obj_id = id(obj)
    if obj_id in measured:
        return 0

    else:

        obj_size = sys.getsizeof(obj)
        measured.add(obj_id)

        if not isinstance(obj, (bytes, bytearray, str)):

            if isinstance(obj, dict):
                obj_size += sum([my_getsizeof(value, measured) + my_getsizeof(key, measured) \
                             for value, key in zip(obj.values(), obj.keys())])

            elif hasattr(obj, '__dict__'):
                obj_size += my_getsizeof(obj.__dict__, measured)

            elif hasattr(obj, '__iter__'):
                obj_size += sum([my_getsizeof(i, measured) for i in obj])

    return obj_size

Примеры запуска функции:

In [8]:
print(my_getsizeof('aaaaaaa'))
print(my_getsizeof(['aaaaaaa']))
print(my_getsizeof(2))
print(my_getsizeof([1,3,5,7]))
print(my_getsizeof(('12','sm',1.4)))
print(my_getsizeof({'code1':'first', 'code2':'second', 'code3':3}))
print(my_getsizeof(range(10)))
print(my_getsizeof(cosmo))

56
128
28
208
198
539
324
1144


### Задача 2 [Многочлены] (0.64 балла)

**Условие:**

Реализовать класс многочлена. Определить операции:

1) *сложения* - **(0.02 балла)**

2) *вычитания* - **(0.02 балла)**

3) *умножения* - **(0.04 балла)**

3a) *быстрого умножения* (алгоритм Карацубы или быстрое преобразование Фурье) - **(+0.25 балла)**

4) *деления* - **(0.05 балла)**

5) *возведения в степень* - **(0.02 балла)** | *возведения в степень* через быстрое возведение в степень за log - **(0.04 балла)**

6) *представления многочлена в человеческом виде* - **(0.02 балла)**

7) *дифференцирования* - **(0.05 балла)**

8) *интегрирования* - **(0.05 балла)**

9) Вызова многочлена как функции (вычисление значения в точке) - **(0.03 балла)**

**Комментарии:**

Для комплексных коэффициентов **(0.01 балла)** к каждому пункту.

Операции с числами также должны работать.

In [9]:
class Poly:
    '''
    An implementation of Polynomial class
    '''

    def __init__(self, *coeff):
        '''
        param coeff: the coeffecients of polynom are listed in order from 0 degree
        type coeff: list of int, float, complex or iterator over int, float, complex
        '''
        if len(list(coeff)) == 0:
            coeff = [0]

        if hasattr(*coeff, '__iter__'):
            self.coeff = list(*coeff)
        else:
            self.coeff = list(coeff)

        for c in self.coeff:
            if not isinstance(c, (int, float, complex)):
                raise TypeError("Wrong number type for", self.coeff)

    def __repr__(self):
        return "Poly((" + str(self.coeff)[1:-1] + "))"

    def degree(self):
        return len(self.coeff) - 1

    def __call__(self, x):
        res = 0
        for i, coeff in enumerate(self.coeff):
            res += coeff * (x**i)
        return res

    def add_zeros_to_array(self, array1, array2):
        len_array1 = len(array1)
        len_array2 = len(array2)
        if len_array1 != len_array2:
            diff_len = abs(len_array1 - len_array2)
            if len_array1 > len_array2:
                array2 += [0] * diff_len
            else:
                array1 += [0] * diff_len
        return array1, array2

    def sign_coef(self, coef, degree):
        if isinstance(coef, complex):
            if coef.real != 0:
                return ' + ' + str(coef)
            else:
                return ' ' + str(coef)

        if degree > 0 and coef == 1:
            return ' + '
        elif degree > 0 and coef == -1:
            return ' - '
        elif coef > 0:
            return ' + ' + str(coef)
        elif coef < 0:
            return ' - ' + str(-coef)
        else:
            return ''

    def x_in_degree(self, degree):
        if degree >= 2:
            return 'x^' + str(degree)
        elif degree == 1:
            return 'x'
        elif degree == 0:
            return ''

    def check_complex_int(self, number):
        if isinstance(number, complex) and number.imag == 0:
            if int(number.real) == number.real:
                number = int(number.real)
            else:
                number = number.real
        return number

    def __str__(self):
        res = ""
        degree = self.degree()

        if degree == 0:
            return str(*self.coeff)

        all_coeffs = self.coeff[::-1]
        coeff_0 = self.check_complex_int(all_coeffs[0])
        count_zeros = 0

        if coeff_0 != 0:

            if coeff_0 == 1:
                res += self.x_in_degree(degree)
            elif coeff_0 == -1:
                res += ' - ' + self.x_in_degree(degree)
            else:
                res += str(coeff_0) + self.x_in_degree(degree)
        else:
            count_zeros += 1

        for i in range(1, len(all_coeffs)):
            coeff = self.check_complex_int(all_coeffs[i])
            if coeff == 0:
                count_zeros += 1
                continue
            else:
                res += self.sign_coef(
                    coeff, degree - i) + self.x_in_degree(degree - i)

        if count_zeros == degree + 1:
            return '0'

        return res.lstrip(' + ')

    def __neg__(self):
        return Poly([x * (-1) for x in self.coeff])

    def __pos__(self):
        return Poly(self.coeff)

    def copy(self):
        return Poly(self.coeff)

    def __add__(self, other):

        if isinstance(self, (int, float, complex)):
            c1 = Poly([self]).coeff
        else:
            c1 = self.coeff

        if isinstance(other, (int, float, complex)):
            c2 = Poly([other]).coeff
        else:
            c2 = other.coeff

        c1, c2 = self.add_zeros_to_array(c1, c2)

        res = [sum(x) for x in zip(c1, c2)]
        return Poly(res)

    __radd__ = __add__

    def __sub__(self, other):
        return self + (-other)

    def __rsub__(self, other):
        return -self + other

    def __mul__(self, other):
        if isinstance(other, (int, float, complex)):
            return Poly([x * other for x in self.coeff])

        multipliers = [0] * (len(self.coeff) + len(other.coeff))
        for i1, c1 in enumerate(self.coeff):
            for i2, c2 in enumerate(other.coeff):
                multipliers[i1 + i2] += c1 * c2
        return Poly(multipliers)

    __rmul__ = __mul__

    def __pow__(self, power):
        if not isinstance(power, int):
            raise TypeError("Wrong number type for", power)
        if power < 0:
            raise ValueError('power should nonnegative')

        res = Poly((1))
        for i in range(power):
            res *= self
        return res

    def derivative(self):
        derivatives = [i * c for i, c in enumerate(self.coeff)]
        derivatives = derivatives[1:]
        return Poly(derivatives)

    def integrate(self, constant=0):
        if not isinstance(constant, (int, float, complex)):
            raise TypeError("Wrong number type for", constant)

        integrals = [c / (i + 1) for i, c in enumerate(self.coeff)]
        return Poly([constant] + integrals)

    def devide_by_number(self, number=1):
        if not isinstance(number, (complex, int, float)):
            raise TypeError("Wrong number type for", number)
        if number == 0:
            raise ValueError("Can not devide by zero")
        return Poly([x / number for x in self.coeff])

Примеры вызова:

Генерация названий класса с аргументами:

In [10]:
print(repr(Poly()))
print(repr(Poly(1)))
print(repr(Poly(1.23)))
print(repr(Poly((-1, 2.34, 1-4j, 7, 0))))
print(repr(Poly([1, -2.04, 4.56, 7, 0, 0, 0])))
print(repr(Poly(range(10))))
print(repr(Poly(map(lambda x: x ** 2, range(5)))))

Poly((0))
Poly((1))
Poly((1.23))
Poly((-1, 2.34, (1-4j), 7, 0))
Poly((1, -2.04, 4.56, 7, 0, 0, 0))
Poly((0, 1, 2, 3, 4, 5, 6, 7, 8, 9))
Poly((0, 1, 4, 9, 16))


Представление многочлена в человеческом виде:

In [11]:
print(Poly())
print(Poly(1.2))
print(Poly([-2,1]))
print(Poly((complex(-1,-3.1),complex(-2,0))))
print(Poly([1.23,0,-6-0.6j,4]))
print(Poly([1.23,0,-6,0]))
print(Poly([-1.23,1-2j,3,1]))
print(Poly(range(-10,1)))
print(Poly(range(11)))
print(Poly([1.2345, 0.0001, -1]))
print(Poly([1, 0, 1, 0, -1, 0, 1]))

0
1.2
x - 2
-2x + (-1-3.1j)
4x^3 + (-6-0.6j)x^2 + 1.23
- 6x^2 + 1.23
x^3 + 3x^2 + (1-2j)x - 1.23
- x^9 - 2x^8 - 3x^7 - 4x^6 - 5x^5 - 6x^4 - 7x^3 - 8x^2 - 9x - 10
10x^10 + 9x^9 + 8x^8 + 7x^7 + 6x^6 + 5x^5 + 4x^4 + 3x^3 + 2x^2 + x
- x^2 + 0.0001x + 1.2345
x^6 - x^4 + x^2 + 1


Вызова многочлена как функции (вычисление значения в точке):

In [12]:
p = Poly((2, 1 - 1.4j, 0, 4, -10, complex(-3, -6.1)))
print('p(x) =', p, '\n')

for x in range(-5, 6):
    print('x =', x, '; p(x) =', p(x))

p(x) = (-3-6.1j)x^5 - 10x^4 + 4x^3 + (1-1.4j)x + 2 

x = -5 ; p(x) = (2622+19069.5j)
x = -4 ; p(x) = (254+6252j)
x = -3 ; p(x) = (-190+1486.5j)
x = -2 ; p(x) = (-96+198j)
x = -1 ; p(x) = (-10+7.5j)
x = 0 ; p(x) = (2+0j)
x = 1 ; p(x) = (-6-7.5j)
x = 2 ; p(x) = (-220-198j)
x = 3 ; p(x) = (-1426-1486.5j)
x = 4 ; p(x) = (-5370-6252j)
x = 5 ; p(x) = (-15118-19069.5j)


Дифференцирование:

In [13]:
p_list = [Poly((1, 2, -3+2j, 4, 0, -55)), Poly((1-1j, 2, 3))] 

for p in p_list:
    print('p(x) =', p)
    print('dp(x)/dx =', p.derivative(), '\n')

p(x) = -55x^5 + 4x^3 + (-3+2j)x^2 + 2x + 1
dp(x)/dx = -275x^4 + 12x^2 + (-6+4j)x + 2 

p(x) = 3x^2 + 2x + (1-1j)
dp(x)/dx = 6x + 2 



Интегрирование:

In [14]:
p_list = [Poly((1, 1-2j, 0, 6, 0, -9)), Poly((1-1j, 2, 3))] 

for p in p_list:
    print('p(x) =', p)
    print('integral p(x) =', p.integrate(0), '\n')

p(x) = -9x^5 + 6x^3 + (1-2j)x + 1
integral p(x) = -1.5x^6 + 1.5x^4 + (0.5-1j)x^2 + x 

p(x) = 3x^2 + 2x + (1-1j)
integral p(x) = x^3 + x^2 + (1-1j)x 



Сложение:

In [15]:
A = Poly(range(5))
B = Poly((1.23, 2.34 - 9j, 0, -5))
C = Poly((0, 0, 0, 0, 0, 0, 2))

print('A + (-A) =', A + (-A))
print('A + B =', A + B)
print('A + C =', A + C)
print('B + C =', B + C)
print('A + B + C', A + B + C)
print('3 + A =', 3 + A)
print('A + 3 =', A + 3)
print('2.1j + B =', 2.1j + B)
print('B + 2.1j =', B + 2.1j)
A += 1
print('A += 1: A =', A)
B += 1.23j
C += B + A
print('B += 1.23j, C += B + A: C =', C)

A + (-A) = 0
A + B = 4x^4 - 2x^3 + 2x^2 + (3.34-9j)x + 1.23
A + C = 2x^6 + 4x^4 + 3x^3 + 2x^2 + x
B + C = 2x^6 - 5x^3 + (2.34-9j)x + 1.23
A + B + C 2x^6 + 4x^4 - 2x^3 + 2x^2 + (3.34-9j)x + 1.23
3 + A = 4x^4 + 3x^3 + 2x^2 + x + 3
A + 3 = 4x^4 + 3x^3 + 2x^2 + x + 3
2.1j + B = - 5x^3 + (2.34-9j)x + (1.23+2.1j)
B + 2.1j = - 5x^3 + (2.34-9j)x + (1.23+2.1j)
A += 1: A = 4x^4 + 3x^3 + 2x^2 + x + 1
B += 1.23j, C += B + A: C = 2x^6 + 4x^4 - 2x^3 + 2x^2 + (3.34-9j)x + (2.23+1.23j)


Вычитание:

In [16]:
A = Poly(range(5))
B = Poly((1.23, 2-4j, 0, -5))
C = Poly((0, 0, 0, 0, 0, 0, 2))

print('A - (+A) =', A - (+A))
print('A - B =', A - B)
print('A - C =', A - C)
print('B - C =', B - C)
print('A - B + C =', A - B + C)
print('3 - A =', 3 - A)
print('A - 3 =', A - 3)
print('2.1 - B =', 2.1j - B)
print('B - 2.1 =', B - 2.1j)
A -= 1
B -= 1.23
C -= B - A
print('A -= 1, B -= 1.23, C -= B - A: C =', C)

A - (+A) = 0
A - B = 4x^4 + 8x^3 + 2x^2 + (-1+4j)x - 1.23
A - C = -2x^6 + 4x^4 + 3x^3 + 2x^2 + x
B - C = -2x^6 - 5x^3 + (2-4j)x + 1.23
A - B + C = 2x^6 + 4x^4 + 8x^3 + 2x^2 + (-1+4j)x - 1.23
3 - A = - 4x^4 - 3x^3 - 2x^2 - x + 3
A - 3 = 4x^4 + 3x^3 + 2x^2 + x - 3
2.1 - B = 5x^3 + (-2+4j)x + (-1.23+2.1j)
B - 2.1 = - 5x^3 + (2-4j)x + (1.23-2.1j)
A -= 1, B -= 1.23, C -= B - A: C = 2x^6 + 4x^4 + 8x^3 + 2x^2 + (-1+4j)x - 1.0


Умножение:

In [17]:
X = Poly((0, 1))
A = Poly(range(5))
B = Poly((1.23, (4 - 9.5j), 0, -5))
C = Poly((0, 0, 0, 0, 0, 0, 2 + 1j))

print('A * B =', A * B)
print('A * C =', A * C)
print('B * C =', B * C)
print('A * (+A) =', A * (+A))
print('A * B * C =', A * B * C)
print('3 * A =', 3 * A)
print('A * 3 =', A * 3)
print('2.1 * B =', (1 + 2.1j) * B)
print('B * 2.1 =', B * (1 + 2.1j))
A *= 1
B *= 1.23
C *= B * A
print('A *= 1, B *= 1.23, C *= B * A: C =', C)
print(
    '7 * X*X*X*X*X*X*X*X*X*X - (3+6j) * X*X*X*X*X + (X - 3) * (2 - X * 5 * X) = ',
    7 * X * X * X * X * X * X * X * X * X * X - (3 + 6j) * X * X * X * X * X +
    (X - 3) * (2 - X * 5 * X))

A * B = - 20x^7 - 15x^6 + (6-38j)x^5 + (11.92-28.5j)x^4 + (11.69-19j)x^3 + (6.46-9.5j)x^2 + 1.23x
A * C = (8+4j)x^10 + (6+3j)x^9 + (4+2j)x^8 + (2+1j)x^7
B * C = (-10-5j)x^9 + (17.5-15j)x^7 + (2.46+1.23j)x^6
A * (+A) = 16x^8 + 24x^7 + 25x^6 + 20x^5 + 10x^4 + 4x^3 + x^2
A * B * C = (-40-20j)x^13 + (-30-15j)x^12 + (50-70j)x^11 + (52.34-45.08j)x^10 + (42.379999999999995-26.310000000000002j)x^9 + (22.42-12.54j)x^8 + (2.46+1.23j)x^7
3 * A = 12x^4 + 9x^3 + 6x^2 + 3x
A * 3 = 12x^4 + 9x^3 + 6x^2 + 3x
2.1 * B = (-5-10.5j)x^3 + (23.95-1.0999999999999996j)x + (1.23+2.583j)
B * 2.1 = (-5-10.5j)x^3 + (23.95-1.0999999999999996j)x + (1.23+2.583j)
A *= 1, B *= 1.23, C *= B * A: C = (-49.2-24.6j)x^13 + (-36.900000000000006-18.450000000000003j)x^12 + (61.5-86.10000000000001j)x^11 + (64.37819999999999-55.4484j)x^10 + (52.127399999999994-32.3613j)x^9 + (27.5766-15.4242j)x^8 + (3.0258+1.5129j)x^7
7 * X*X*X*X*X*X*X*X*X*X - (3+6j) * X*X*X*X*X + (X - 3) * (2 - X * 5 * X) =  7x^10 + (-3-6j)x^5 - 5x^3 + 15x^2 + 

Возведение в степень:

In [18]:
X = Poly((0, 1))
A = Poly((1, 1))
B = Poly((2j))
C = Poly((1, 0, 0, 0, 0, 0, 0, 0, 1 - 2.34j, 0, 0, 0, -2))

print('A ** 0 =', A**0)
print('A ** 1 =', A**1)
print('A ** 2 =', A**2)
print('A ** 10 =', A**10)
print('B ** 100 =', B**100)
print('C**3 =', C**3)
print('7 * X**10 - 3 * X**5 + (X - 3) * (2 - 5 * X**2) =',
      7 * X**10 - 3 * X**5 + (X - 3) * (2 - 5 * X**2))

A ** 0 = 1
A ** 1 = x + 1
A ** 2 = x^2 + 2x + 1
A ** 10 = x^10 + 10x^9 + 45x^8 + 120x^7 + 210x^6 + 252x^5 + 210x^4 + 120x^3 + 45x^2 + 10x + 1
B ** 100 = 1267650600228229401496703205376
C**3 = - 8x^36 + (12-28.08j)x^32 + (26.853599999999993+28.08j)x^28 + (-3.4267999999999965+5.792903999999998j)x^24 + (-12+28.08j)x^20 + (-13.426799999999997-14.04j)x^16 - 6x^12 + (3-7.02j)x^8 + 1
7 * X**10 - 3 * X**5 + (X - 3) * (2 - 5 * X**2) = 7x^10 - 3x^5 - 5x^3 + 15x^2 + 2x - 6


Деление на число:

In [19]:
A = Poly((1, 0, 0, 0, 0, 0, 0, 0, 1 - 2.34j, 0, 0, 0, -2))

print('A / 2 =', A.devide_by_number(2))
print('A / (1+10j) =', A.devide_by_number(1+10j))
print('A / (-100) =', A.devide_by_number(-100))
print('A / 0 =', A.devide_by_number(0))

A / 2 = - x^12 + (0.5-1.17j)x^8 + 0.5
A / (1+10j) = (-0.019801980198019802+0.19801980198019803j)x^12 + (-0.22178217821782176-0.12217821782178218j)x^8 + (0.009900990099009901-0.09900990099009901j)
A / (-100) = 0.02x^12 + (-0.01+0.023399999999999997j)x^8 - 0.01


ValueError: Can not devide by zero

### Задача 3 [Аналог range] (0.05 балла)

**Условие:**

Реализуйте итератор с поведением, аналогичным range.

In [20]:
def my_range(*args):
    '''
    An analog of function range
    '''
    for arg in args:
        if not isinstance(arg, int):
            raise ValueError("Wrong number type for %s " % arg)
            
    if len(args) == 1:
        start = 0
        stop = args[0] 
        step = 1
        
    elif len(args) == 2:
        start = args[0] 
        stop = args[1] 
        step = 1 
        
    elif len(args) == 3: 
        start = args[0] 
        stop = args[1] 
        step = args[2] 
        
    else:
        raise ValueError("Wrong number of arguments")

    i = start
    
    if step > 0:
        while i < stop:
            yield i
            i += step
    else:
        while i > stop:
            yield i
            i += step

Примеры запусков:

In [21]:
print(list(my_range(-7,-3)))
print(list(my_range(1, 10, 2)))
print(list(my_range(0, 7,-1)))
print(list(my_range(-7)))
print(list(my_range(-7, '23')))

[-7, -6, -5, -4]
[1, 3, 5, 7, 9]
[]
[]


ValueError: Wrong number type for 23 

### Задача 4 [Primary Key] (0.05 балла)

**Условие:**

С помощью механизма дескрипторов реализуйте Primary Key - свойства первичного ключа из PostgreSQL.

In [22]:
class PrimaryKey:
    '''
    A discriptor for class Primary Key
    '''
    def __init__(self, name):
        self.name = name
        self.primarykey = []
        self.primarykey_len = None
        self.primarykey_types = []
        
    def __get__(self, instance, owner):
        return self.primarykey

    def check_primarykey_attrs(self, new_key):

        if not isinstance(new_key, (int, float, str, list)):
            raise TypeError("Wrong type for PrimaryKey")
        
        newkey_len = len(new_key) if isinstance(new_key, list) else 1
        
        newkey_types = []

        if newkey_len > 1:
            for subkey in new_key:
                newkey_types.append(type(subkey))
        else: 
            if type(new_key) == list:
                newkey_types.append(type(new_key[0]))
            else:
                newkey_types = type(new_key)

        if len(self.primarykey) == 0:
            self.primarykey_len = newkey_len
            self.primarykey_types = newkey_types

        if newkey_len != self.primarykey_len:
            raise ValueError("PrimaryKey should contain the same number of elements ({})".format(self.primarykey_len))
        
        if newkey_types != self.primarykey_types:
            raise TypeError("PrimaryKey should be of type {}".format(self.primarykey_types))

    
    def __set__(self, instance, new_key):

        self.check_primarykey_attrs(new_key)
                    
        if new_key not in self.primarykey:
            if new_key != None:
                instance.__dict__[self.name] = new_key
                self.primarykey.append(new_key)
            else:
                raise ValueError("PrimaryKey should not be None")
        else:
            raise ValueError("PrimaryKey should have unique values")
                
    def __delete__(self, instance):
        self.primarykey.remove(instance.__dict__[self.name])
        del instance

In [23]:
class TableWithTwoColumnsPrimaryField:
    '''
    A class of field of table with two columns, one of which is PrimaryKey
    '''
    primarykeycolumn = PrimaryKey("primarykey")
    
    def __init__(self, primarykey, column):
        self.primarykeycolumn = primarykey
        self.column = column

    def __del__(self):
        del self.primarykeycolumn


y1 = TableWithTwoColumnsPrimaryField([2,"2"], "one")
y2 = TableWithTwoColumnsPrimaryField([3, "5"], "two")
y3 = TableWithTwoColumnsPrimaryField([1,'4'], "three")

print(y1.primarykey)
print(y2.primarykey)
print(y3.primarykey)

del y2

y4 = TableWithTwoColumnsPrimaryField([2,'2'], "two 2")

[2, '2']
[3, '5']
[1, '4']


ValueError: PrimaryKey should have unique values

### Задача 5 [PositiveSmallIntegerField] (0.03 балла)

**Условие:**

С помощью механизма дескрипторов реализуйте тип данных PositiveSmallIntegerField - поле, принимающее значения от 0 до 32767.

In [24]:
class DescPositiveSmallIntegerField:
    '''
    A discriptor for small positive interger numbers in range [0, 32767]
    '''
    def __init__(self, initval=None):
        self.val = initval

    def __get__(self, instance, objtype):
        return self.val

    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise TypeError("Wrong number type for value")
        if value >= 0 and value <= 32767:
            self.val = value
        else:
            raise ValueError("Values should be in range [0, 32767]")
                
    def __delete__(self, instance):
        del instance

In [25]:
class PositiveSmallIntegerField:
    '''
    A class of small positive interger numbers in range [0, 32767]
    '''
    number = DescPositiveSmallIntegerField()
    
    def __init__(self, number):
        self.number = number

Примеры запуска:

In [26]:
a = PositiveSmallIntegerField(832)
# PositiveSmallIntegerField(-3).number
# PositiveSmallIntegerField(3.).number
# PositiveSmallIntegerField('3').number

In [27]:
del a

In [28]:
a

NameError: name 'a' is not defined

### Задача 6 [Timer] (0.02 балла)

**Условие:**

Реализовать контекстный менеджер, который выводит время, проведенное в нём.

In [30]:
import time

class measure_time:
    '''
    A context manager measures time spent in it
    '''
    def __enter__(self):
        self.start_time = time.time()
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end_time = time.time()
        self.time = (self.end_time - self.start_time) * 1000
        print('Functions in context manager finished in {} ms'.format(self.time))
        if exc_val:
            raise

Пример запуска:

In [31]:
with measure_time() as mt:
    res = [0] * 1000000

Functions in context manager finished in 5.6247711181640625 ms
